In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

# Загрузка данных
data = np.load('images_r.npy')
labels = np.load('masks_r.npy')

# Разбиение данных на train и test
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

# Преобразование данных в тензоры
train_data = torch.Tensor(train_data)
train_labels = torch.Tensor(train_labels)
test_data = torch.Tensor(test_data)
test_labels = torch.Tensor(test_labels)

# Определение модели
import torch.nn.functional as F

class ImprovedEncoderDecoder(nn.Module):
    def __init__(self):
        super(ImprovedEncoderDecoder, self).__init__()

        # Энкодер
        self.encoder_conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.encoder_conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.encoder_conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.encoder_conv4 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.dropout1 = nn.Dropout2d(p=0.2)
        self.encoder_pool = nn.MaxPool2d(2, 2)

        # Декодер
        self.decoder_upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.decoder_conv1 = nn.Conv2d(128, 64, 3, padding=1)
        self.bn5 = nn.BatchNorm2d(64)
        self.decoder_conv2 = nn.Conv2d(64, 32, 3, padding=1)
        self.bn6 = nn.BatchNorm2d(32)
        self.decoder_conv3 = nn.Conv2d(32, 16, 3, padding=1)
        self.bn7 = nn.BatchNorm2d(16)
        self.dropout2 = nn.Dropout2d(p=0.2)
        self.decoder_conv4 = nn.Conv2d(16, 1, 3, padding=1)


    def forward(self, x):
        # Энкодер
        x = F.relu(self.bn1(self.encoder_conv1(x)))
        x = F.relu(self.bn2(self.encoder_conv2(x)))
        x = self.encoder_pool(F.relu(self.bn3(self.encoder_conv3(x))))
        x = self.dropout1(x)
        x = self.encoder_pool(F.relu(self.bn4(self.encoder_conv4(x))))

        # Декодер
        x = self.decoder_upsample(x)
        x = F.relu(self.bn5(self.decoder_conv1(x)))
        x = self.decoder_upsample(x)
        x = F.relu(self.bn6(self.decoder_conv2(x)))
        x = F.relu(self.bn7(self.decoder_conv3(x)))
        x = self.dropout2(x)
        x = torch.sigmoid(self.decoder_conv4(x))

        return x





# Создание экземпляра модели и определение функции потерь и оптимизатора
model = ImprovedEncoderDecoder()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Обучение модели
# Обучение модели
batch_size = 16
epochs = 15
num_batches = len(train_data) // batch_size
for epoch in range(epochs):
    running_loss = 0.0
    running_recall = 0.0
    running_precision = 0.0
    for i in range(num_batches):
        # Обнуление градиентов перед каждой итерацией обучения
        optimizer.zero_grad()

        # Прямой проход
        batch_data = train_data[i*batch_size:(i+1)*batch_size]
        batch_labels = train_labels[i*batch_size:(i+1)*batch_size]
        outputs = model(batch_data.unsqueeze(1))
        loss = criterion(outputs, batch_labels.unsqueeze(1))
        running_loss += loss.item()

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

        # Расчет метрик recall и precision
        with torch.no_grad():
            predicted = (outputs > 0.5).float()
            true_positives = (predicted * batch_labels.unsqueeze(1)).sum()
            false_positives = (predicted * (1 - batch_labels.unsqueeze(1))).sum()
            false_negatives = ((1 - predicted) * batch_labels.unsqueeze(1)).sum()
            recall = true_positives / (true_positives + false_negatives)
            precision = true_positives / (true_positives + false_positives)
            running_recall += recall.item()
            running_precision += precision.item()

    # Вывод loss, recall и precision на каждой эпохе
    print(f"Epoch {epoch + 1} loss: {running_loss / num_batches}, recall: {running_recall / num_batches}, precision: {running_precision / num_batches}")

# Тестирование модели
# Тестирование модели
with torch.no_grad():
    model.eval()
    test_outputs = (model(test_data.unsqueeze(1)) > 0.5).float()
    test_loss = criterion(test_outputs, test_labels.unsqueeze(1))
    print(f"Test loss: {test_loss.item()}")
    batch_size = 16
    num_batches = len(test_data) // batch_size
    running_recall = 0.0
    running_precision = 0.0
    false_positive_pixels = 0  # счетчик пикселей, на которых был ненастоящий сигнал на входных данных, но на выходных данных его нет
    false_positive_total_pixels = 0  # общее количество пикселей с ненастоящим сигналом на входных данных
    for i in range(num_batches):
        batch_data = test_data[i * batch_size:(i + 1) * batch_size]
        batch_labels = test_labels[i * batch_size:(i + 1) * batch_size]
        outputs = model(batch_data.unsqueeze(1))
        predicted = (outputs > 0.5).float()
        true_positives = (predicted * batch_labels.unsqueeze(1)).sum()
        false_positives = (predicted * (1 - batch_labels.unsqueeze(1))).sum()
        false_negatives = ((1 - predicted) * batch_labels.unsqueeze(1)).sum()
        recall = true_positives / (true_positives + false_negatives)
        precision = true_positives / (true_positives + false_positives)
        running_recall += recall.item()
        running_precision += precision.item()
        false_positive_pixels += ((batch_data == 1) & (predicted == 0)).sum()
        false_positive_total_pixels += ((batch_data == 1) & (batch_labels == 0)).sum()


    print(f"Test recall: {running_recall / num_batches}, Test precision: {running_precision / num_batches}")
    false_positive_rate = false_positive_pixels / false_positive_total_pixels
    print(f"False positive rate: {false_positive_rate}")
import matplotlib.pyplot as plt

total_true_removed_signals = 0
total_predicted_removed_signals = 0
total_true_signals = 0
total_input_true_removed_signals = 0

for i in range(len(test_data)):
    # plt.matshow(test_data[i].cpu(), aspect='auto')
    # plt.title("Input")
    # plt.show()

    # plt.matshow(test_labels[i].cpu(), aspect='auto')
    # plt.title("True Mask")
    # plt.show()

    # plt.matshow((test_outputs[i, 0] > 0.5).float().cpu(), aspect='auto')
    # plt.title("Predicted Mask")
    # plt.show()

    input_signal = test_data[i].cpu().numpy()
    true_mask = test_labels[i].cpu().numpy()
    predicted_mask = (test_outputs[i, 0] > 0.5).float().cpu().numpy()

    # Calculate the number of signals on each line
    print(f"Number of signals on each line:")
    for j in range(input_signal.shape[0]):
        line_input_signals = 0
        line_true_removed_signals = 0
        line_predicted_removed_signals = 0

        # Track the start of a continuous sequence of 1s
        start = -1
        for k in range(input_signal.shape[1]):
            if input_signal[j, k] == 1:
                if start == -1:
                    start = k
            else:
                if start != -1:
                    line_input_signals += 1
                    if np.all(true_mask[j, start:k] == 0):
                        line_true_removed_signals += 1
                        if np.all(predicted_mask[j, start:k] == 0):
                            line_predicted_removed_signals += 1
                    start = -1

        # Check if there is an ongoing sequence at the end
        if start != -1:
            line_input_signals += 1
            if np.all(true_mask[j, start:] == 0):
                line_true_removed_signals += 1
                if np.all(predicted_mask[j, start:] == 0):
                    line_predicted_removed_signals += 1

        print(f"Line {j+1}: Input Signals: {line_input_signals}, True Removed Signals: {line_true_removed_signals}, Predicted Removed Signals: {line_predicted_removed_signals}")

        # Accumulate the total number of True Removed Signals and Predicted Removed Signals
        total_true_removed_signals += line_true_removed_signals
        total_predicted_removed_signals += line_predicted_removed_signals

        # Accumulate the total number of True Signals where input=1, true=1, and predicted=0
        total_input_true_removed_signals += np.sum((input_signal[j, :] == 1) & (true_mask[j, :] == 1) & (predicted_mask[j, :] == 0))

    # Calculate the total number of True Signals
    total_true_signals += np.sum(true_mask == 1)

    print()

# Calculate the percentage of Predicted Removed Signals over True Removed Signals
percent_removed_predicted_true = (total_predicted_removed_signals / total_true_removed_signals) * 100
percent_removed_input_true = (total_input_true_removed_signals / total_true_signals) * 100

print(f"Total True Removed Signals: {total_true_removed_signals}")
print(f"Total Predicted Removed Signals: {total_predicted_removed_signals}")
print(f"Percentage of Predicted Removed Signals / True Removed Signals: {percent_removed_predicted_true}%")
print(f"Percentage of Input True Signals Removed / True Signals: {percent_removed_input_true}%")
